# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kingston,17.9970,-76.7936,30.09,74,75,5.66,JM,1732905772
1,1,nar'yan-mar,67.6713,53.0870,-3.67,93,100,7.99,RU,1732905774
2,2,edinburgh of the seven seas,-37.0676,-12.3116,16.06,90,100,6.33,SH,1732905641
3,3,waitangi,-43.9535,-176.5597,11.01,77,97,5.81,NZ,1732905776
4,4,iqaluit,63.7506,-68.5145,-7.15,79,100,1.54,CA,1732905777


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',  # Longitude column
    y='Lat',  # Latitude column
    geo=True,  # Enable geo plotting for map projection
    size='Humidity',  # Size of the points based on the Humidity column
    color='City',  # Color the points by City for differentiation
    hover_cols=['City', 'Humidity', 'Lat', 'Lng'],  # Columns to display on hover
    size_max=50,  # Max size of the point
    tiles='OSM',  # OpenStreetMap tiles for the background
    title="City Locations with Humidity Levels"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp = city_data_df.loc[(city_data_df['Max Temp'] >= 30) & (city_data_df['Max Temp'] <= 100)]  # Temperature between 30°F and 100°F
city_humidity = city_temp.loc[city_temp['Humidity'] > 60]  # Humidity more than 60%
city_cloudiness = city_humidity.loc[city_humidity['Cloudiness'] > 30]  # Cloudiness more than 30%
city_wind_speed = city_cloudiness.loc[city_cloudiness['Wind Speed'] < 10]  # Wind Speed below 10 mph

# Drop any rows with null values
ideal_weather_df = city_wind_speed.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kingston,17.9970,-76.7936,30.09,74,75,5.66,JM,1732905772
14,14,road town,18.4167,-64.6167,30.02,74,40,3.09,VG,1732905789
238,238,juan griego,11.0817,-63.9655,30.77,63,54,6.60,VE,1732906080
282,282,sao joao da barra,-21.6403,-41.0511,33.05,85,66,5.78,BR,1732906140
541,541,cartagena,10.3997,-75.5144,31.79,70,40,6.69,CO,1732906484


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kingston,JM,17.9970,-76.7936,74,
14,road town,VG,18.4167,-64.6167,74,
238,juan griego,VE,11.0817,-63.9655,63,
282,sao joao da barra,BR,-21.6403,-41.0511,85,
541,cartagena,CO,10.3997,-75.5144,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000  # Set the radius to search within (10 km)

params = {
    "apiKey": geoapify_key,  # Use the Geoapify API key
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1  # Limit to 1 hotel result per search
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"  # Define a circular search area around the city
    params["bias"] = f"proximity:{lon},{lat}"  # Bias the results based on the city coordinates

    # Set base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data with the updated hotel names
hotel_df

Starting hotel search
kingston - nearest hotel: Aphrodite Hotel
road town - nearest hotel: No hotel found
juan griego - nearest hotel: Hotel Descanso Tropical
sao joao da barra - nearest hotel: Pousada Mediterrâneo
cartagena - nearest hotel: Hotel Bellamar


,City,Country,Lat,Lng,Humidity,Hotel Name
0,kingston,JM,17.9970,-76.7936,74,Aphrodite Hotel
14,road town,VG,18.4167,-64.6167,74,No hotel found
238,juan griego,VE,11.0817,-63.9655,63,Hotel Descanso Tropical
282,sao joao da barra,BR,-21.6403,-41.0511,85,Pousada Mediterrâneo
541,cartagena,CO,10.3997,-75.5144,70,Hotel Bellamar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',  # Longitude column
    y='Lat',  # Latitude column
    geo=True,  # Enable geo plotting for map projection
    size='Humidity',  # Size of the points based on the Humidity column
    color='City',  # Color the points by City for differentiation
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity', 'Lat', 'Lng'],  # Add 'Hotel Name' and 'Country' to the hover message
    size_max=50,  # Max size of the point
    tiles='OSM',  # OpenStreetMap tiles for the background
    title="City Locations with Hotel and Humidity Information"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)